This program ensures that each .CDRx file has an accompanying .CSV file containing all of that file's information.

Because there is no 'decoder' for information in .CDRx files, we can leverage the 'export to CSV' function in the Bosch Crash Data Retrieval software.

In [133]:
import os
import pandas as pd
from m0_1_system_variables import raw_data_path

In [141]:
#This block of code creates a list of paths to cdrx files which do not have an accompanying .CSV file. 
#The objective of this module is to find all of these 'unaccompanied' files and then create a .CSV version suitable for import and analysis. 


#Whenever the database is re-scanned for new files, the unaccompanied_cdrx_path_list must be cleared. Otherwise, it will simply add the new list of unaccompanied files to an old list.

unaccompanied_cdrx_path_list = []
complete_cdrx_path_list = []

for root, dirs, files in os.walk(raw_data_path):
        for filename in files:
                if filename.lower().endswith('cdrx'):
                        #Ford JH trucks are not supported. Need to exclude via VIN.
                        # if filename.startswith
                                complete_cdrx_path_list.append(r"{0}\{1}".format(root, filename))
                                csv_filename = os.path.splitext(filename)[0] + '.CSV'
                                if csv_filename not in files:     
                                # os.path.join is performed to handle the case of one vehicle having multiple records. 
                                # The record is saved as the vehicle's VIN, but successive records bearing the same name are placed in different directories.
                                # Considering the filepath as part of the record's 'identity' will ensure that these edge cases are accommodated.

                                        # unaccompanied_cdrx_path_list.append(os.path.join(root, filename))
                                        #Updating this to raw string for 'cleanliness' and resilience. This prevents any need to escape the backslashes.
                                        unaccompanied_cdrx_path_list.append(r"{0}\{1}".format(root, filename))



dfs = []

for file_path in complete_cdrx_path_list:

        df = pd.DataFrame({'filepath': [file_path]})

        # Append the DataFrame to the list of DataFrames
        dfs.append(df)
# Combine the list of DataFrames into a single DataFrame
complete_path_list_dataframe = pd.concat(dfs, ignore_index=True)

In [145]:
unaccompanied_cdrx_path_list
#Noticed here that many files are leftover after batch conversion. These files cannot be exported to CSV in Bosch.
#These leftover files, if significant, could be exported to PDF and scraped another way.

#Upon manual inspection, these files which could not be exported appear to be 2014 GMC trucks, and JH models.

['C:\\Code\\MFG598\\final_project\\test_cdr_directory\\1GDY72CA8E1916159_ACM\\1GDY72CA8E1916159_ACM.CDRx',
 'C:\\Code\\MFG598\\final_project\\test_cdr_directory\\1GDY72CA9E1916994_ACM\\1GDY72CA9E1916994_ACM.CDRx',
 'C:\\Code\\MFG598\\final_project\\test_cdr_directory\\1GTN1TEC0FZ904779_ACMA\\1GTN1TEC0FZ904779_ACMA.CDRx',
 'C:\\Code\\MFG598\\final_project\\test_cdr_directory\\1GTN1TEC0FZ904779_ACMB\\1GTN1TEC0FZ904779_ACMB.CDRx',
 'C:\\Code\\MFG598\\final_project\\test_cdr_directory\\2GCEC19TX41181246_ACM\\2GCEC19TX41181246_ACM.CDRx',
 'C:\\Code\\MFG598\\final_project\\test_cdr_directory\\3GCNAAED6NG662964_ACM\\3GCNAAED6NG662964_ACM.CDRx']

In [142]:
print(len(unaccompanied_cdrx_path_list))

6


This block is used to determine whether the Bosch CDR tool is running

In [139]:
from m0_1_system_variables import bosch_filepath
import psutil
import sys
import subprocess
import time
import pyautogui as gui
import pyperclip as pyp

def check_process(process_name):
    """Check if a process is running on Windows"""
    for proc in psutil.process_iter(['name']):
        if proc.name() == process_name:
            return True
    return False
    sys.exit(1)

    # Check if 'cdr.exe' is running
if check_process('CDR.EXE'):
    print('CDR.EXE is running. Killing CDR.EXE and re-starting process to make it visible')
    subprocess.run(['taskkill', '/f', '/im', 'cdr.exe'])
    time.sleep(0.5)
    process = subprocess.Popen(bosch_filepath)
    time.sleep(1)
else:
    process = subprocess.Popen(bosch_filepath)
    time.sleep(1)

#Must truncate the filepath for use in CDR to eliminate the filename and last backslash.
for working_file in unaccompanied_cdrx_path_list:
    working_file_str = str(working_file)
    #opens the CDR 'open file' menu and explorer
    gui.hotkey('ctrl', 'o')
    time.sleep(0.1)
    pyp.copy(working_file_str)
    #paste filepath and name into 'name' toolbar.
    gui.hotkey('ctrl','v')
    gui.press('enter')
    #time delay to load file
    time.sleep(5)
    gui.press('alt')
    gui.press('right')
    for i in range(9):
        gui.press('down')
    #Select 'save as CSV'
    gui.press('enter')
    #Saves the file
    gui.press('enter')
    time.sleep(0.2)


This code scans the file structure and deletes any corrupt files that have 0kb data. Several of these were found during the batch conversion process on the test database.

In [140]:
import os

# Set the path to the directory you want to scan
from m0_1_system_variables import raw_data_path

for root, dir, filenames in os.walk(raw_data_path):
    for filename in filenames:

        #Check if the file ends with ".CDRx" and its size is 0KB
        if os.path.getsize(os.path.join(root, filename)) == 0:
            # Delete the file
            os.remove(os.path.join(root, filename))
            print(f'Deleted file {filename}')